## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-02-18-17-14-06 +0000,nypost,Airbnb looking for new hosts in World Cup citi...,https://nypost.com/2026/02/18/business/airbnb-...
1,2026-02-18-17-11-52 +0000,bbc,Zuckerberg arrives to testify in social media ...,https://www.bbc.com/news/articles/c5y42znjnjvo...
2,2026-02-18-17-06-31 +0000,nypost,Wild video shows avalanche spill down mountain...,https://nypost.com/2026/02/18/world-news/wild-...
3,2026-02-18-17-05-02 +0000,nypost,Texas teen awarded $3.2M jury verdict after mo...,https://nypost.com/2026/02/18/us-news/texas-te...
4,2026-02-18-17-00-27 +0000,nyt,"Fritz Lang’s ‘Nibelungen,’ the Way It Was Mean...",https://www.nytimes.com/2026/02/18/arts/music/...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2482/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
280,trump,28
2,new,21
889,jackson,17
888,jesse,15
341,talks,15


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
50,2026-02-18-15-33-00 +0000,wsj,An immigration court dropped the Trump adminis...,https://www.wsj.com/politics/policy/judge-drop...,106
123,2026-02-18-11-59-00 +0000,wsj,Stephen Colbert’s stinging criticism of Presid...,https://www.wsj.com/politics/how-stephen-colbe...,104
142,2026-02-18-11-00-00 +0000,nypost,Lara Trump reveals Trump’s unexpected reaction...,https://nypost.com/2026/02/18/us-news/lara-tru...,94
234,2026-02-18-00-48-22 +0000,bbc,"Under pressure from Trump, Venezuela's new pre...",https://www.bbc.com/news/articles/cn87rv0jdy1o...,92
362,2026-02-17-17-49-41 +0000,nypost,Stephen Colbert rips CBS for pulling interview...,https://nypost.com/2026/02/17/media/colbert-ri...,92


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
50,106,2026-02-18-15-33-00 +0000,wsj,An immigration court dropped the Trump adminis...,https://www.wsj.com/politics/policy/judge-drop...
353,76,2026-02-17-18-14-06 +0000,nyt,Ukraine and Russia Hold New Round of Peace Tal...,https://www.nytimes.com/2026/02/17/world/europ...
123,76,2026-02-18-11-59-00 +0000,wsj,Stephen Colbert’s stinging criticism of Presid...,https://www.wsj.com/politics/how-stephen-colbe...
245,59,2026-02-18-00-23-43 +0000,bbc,'He did it for us': US soldier recalls Jesse J...,https://www.bbc.com/news/articles/cwykn2endnyo...
84,53,2026-02-18-14-07-23 +0000,nypost,Sheriff leading Nancy Guthrie probe admits to ...,https://nypost.com/2026/02/18/us-news/nancy-gu...
202,45,2026-02-18-03-06-25 +0000,cbc,Rescuers push through winter storm to 6 surviv...,https://www.cbc.ca/news/world/california-avala...
188,44,2026-02-18-04-50-10 +0000,bbc,Suspect in Rhode Island ice hockey shooting ki...,https://www.bbc.com/news/articles/cy4wvk9vn40o...
142,38,2026-02-18-11-00-00 +0000,nypost,Lara Trump reveals Trump’s unexpected reaction...,https://nypost.com/2026/02/18/us-news/lara-tru...
70,34,2026-02-18-14-53-01 +0000,nypost,Texas Dem Senate candidate James Talarico rake...,https://nypost.com/2026/02/18/us-news/texas-de...
354,33,2026-02-17-18-11-11 +0000,wapo,"Amid threat of U.S. attack, Iran nuclear talks...",https://www.washingtonpost.com/world/2026/02/1...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
